# Análise Exploratória de Dados - Olist E-Commerce

**Objetivo:** Análise exploratória completa do dataset Olist para entender padrões, distribuições e qualidade dos dados.

---

## Índice

1. [Setup e Configuração](#1-setup)
2. [Carregamento dos Dados](#2-load)
3. [Análise de Qualidade](#3-quality)
4. [Análise Univariada](#4-univariate)
5. [Análise Bivariada](#5-bivariate)
6. [Análise Temporal](#6-temporal)
7. [Análise Geográfica](#7-geo)
8. [Insights e Conclusões](#8-insights)

## 1. Setup e Configuração

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from datetime import datetime, timedelta
import warnings
from dotenv import load_dotenv
import os

warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuração do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Carregar variáveis de ambiente
load_dotenv()

# Configuração BigQuery
PROJECT_ID = os.getenv('GCP_PROJECT_ID')
DATASET_ID = os.getenv('GCP_DATASET_ID', 'olist_ecommerce')

# Cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Criar diretório para imagens
os.makedirs('../docs/images', exist_ok=True)

print(f"✓ Setup completo")
print(f"✓ Projeto: {PROJECT_ID}")
print(f"✓ Dataset: {DATASET_ID}")
print(f"✓ Diretório de imagens criado")

## 2. Carregamento dos Dados

Vamos carregar dados diretamente do BigQuery usando as views já criadas.

In [ ]:
def query_bigquery(query: str) -> pd.DataFrame:
    """Helper para executar queries no BigQuery"""
    return client.query(query).to_dataframe()

# Carregar view principal de pedidos
query_orders = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.vw_orders_complete`
WHERE order_status = 'delivered'
LIMIT 50000
"""

df_orders = query_bigquery(query_orders)

print(f"✓ {len(df_orders):,} pedidos carregados")
print(f"✓ Período: {df_orders['order_purchase_timestamp'].min()} a {df_orders['order_purchase_timestamp'].max()}")

In [ ]:
# Carregar itens de pedidos
query_items = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.vw_order_items_detail`
LIMIT 50000
"""

df_items = query_bigquery(query_items)

print(f"✓ {len(df_items):,} itens carregados")

## 3. Análise de Qualidade dos Dados {#3-quality}

In [ ]:
# Estatísticas gerais
print("="*60)
print("ESTATÍSTICAS GERAIS")
print("="*60)

print(f"\nPedidos:")
print(f"  Total: {len(df_orders):,}")
print(f"  Clientes únicos: {df_orders['customer_unique_id'].nunique():,}")
print(f"  Período: {(df_orders['order_purchase_timestamp'].max() - df_orders['order_purchase_timestamp'].min()).days} dias")

print(f"\nReceita:")
print(f"  Total: R$ {df_orders['payment_value'].sum():,.2f}")
print(f"  Ticket médio: R$ {df_orders['payment_value'].mean():.2f}")
print(f"  Mediana: R$ {df_orders['payment_value'].median():.2f}")

In [ ]:
# Análise de missing values
print("\n" + "="*60)
print("MISSING VALUES")
print("="*60)

missing = df_orders.isnull().sum()
missing_pct = (missing / len(df_orders)) * 100
missing_df = pd.DataFrame({
    'Missing': missing,
    'Percentage': missing_pct
}).sort_values('Missing', ascending=False)

print(missing_df[missing_df['Missing'] > 0])

In [ ]:
# Análise de duplicatas
duplicates = df_orders.duplicated(subset=['order_id']).sum()
print(f"\nPedidos duplicados: {duplicates}")

if duplicates > 0:
    print("Atenção: Existem pedidos duplicados!")
else:
    print("Nenhum pedido duplicado encontrado")

## 4. Análise Univariada

### 4.1 Distribuição de Receita

In [ ]:
plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histograma de valores
axes[0, 0].hist(df_orders['payment_value'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Valor do Pedido (R$)')
axes[0, 0].set_ylabel('Frequência')
axes[0, 0].set_title('Distribuição de Valores de Pedidos')
axes[0, 0].axvline(df_orders['payment_value'].mean(), color='red', linestyle='--', label='Média')
axes[0, 0].axvline(df_orders['payment_value'].median(), color='green', linestyle='--', label='Mediana')
axes[0, 0].legend()

# Boxplot
axes[0, 1].boxplot(df_orders['payment_value'], vert=True)
axes[0, 1].set_ylabel('Valor do Pedido (R$)')
axes[0, 1].set_title('Boxplot - Valores de Pedidos')

# Log scale
axes[1, 0].hist(np.log1p(df_orders['payment_value']), bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Log(Valor do Pedido + 1)')
axes[1, 0].set_ylabel('Frequência')
axes[1, 0].set_title('Distribuição (Escala Log)')

# Estatísticas descritivas
stats_text = f"""Estatísticas Descritivas:
Média: R$ {df_orders['payment_value'].mean():.2f}
Mediana: R$ {df_orders['payment_value'].median():.2f}
Desvio Padrão: R$ {df_orders['payment_value'].std():.2f}
Mínimo: R$ {df_orders['payment_value'].min():.2f}
Máximo: R$ {df_orders['payment_value'].max():.2f}
P25: R$ {df_orders['payment_value'].quantile(0.25):.2f}
P75: R$ {df_orders['payment_value'].quantile(0.75):.2f}
P90: R$ {df_orders['payment_value'].quantile(0.90):.2f}
"""
axes[1, 1].text(0.1, 0.5, stats_text, fontsize=12, verticalalignment='center', family='monospace')
axes[1, 1].axis('off')
axes[1, 1].set_title('Estatísticas Descritivas')

plt.tight_layout()
try:
    plt.savefig('../docs/images/eda_revenue_distribution.png', dpi=300, bbox_inches='tight')
    print("✓ Gráfico salvo: docs/images/eda_revenue_distribution.png")
except Exception as e:
    print(f"Erro ao salvar gráfico: {e}")
plt.show()

### 4.2 Distribuição de Review Scores (NPS)

In [ ]:
plt.close('all')
# Análise de NPS
nps_counts = df_orders['review_score'].value_counts().sort_index()
nps_pct = (nps_counts / nps_counts.sum()) * 100

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Barplot de contagens
axes[0].bar(nps_counts.index, nps_counts.values, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Review Score')
axes[0].set_ylabel('Quantidade')
axes[0].set_title('Distribuição de Review Scores')
axes[0].set_xticks([1, 2, 3, 4, 5])

# Pie chart
colors = ['#ff4444', '#ff8844', '#ffbb44', '#88dd44', '#44dd44']
axes[1].pie(nps_pct.values, labels=nps_pct.index, autopct='%1.1f%%', colors=colors, startangle=90)
axes[1].set_title('Distribuição Percentual de Review Scores')

plt.tight_layout()
try:
    plt.savefig('../docs/images/eda_nps_distribution.png', dpi=300, bbox_inches='tight')
    print("✓ Gráfico salvo: docs/images/eda_nps_distribution.png")
except Exception as e:
    print(f"Erro ao salvar gráfico: {e}")
plt.show()

print(f"\nNPS Médio: {df_orders['review_score'].mean():.2f}")
print(f"NPS Positivo (4-5): {((nps_counts[4] + nps_counts[5]) / nps_counts.sum() * 100):.1f}%")
print(f"NPS Negativo (1-2): {((nps_counts[1] + nps_counts[2]) / nps_counts.sum() * 100):.1f}%")

## 5. Análise Bivariada

### 5.1 Valor do Pedido vs Review Score

In [ ]:
plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Boxplot: Valor por Review Score
df_orders.boxplot(column='payment_value', by='review_score', ax=axes[0])
axes[0].set_xlabel('Review Score')
axes[0].set_ylabel('Valor do Pedido (R$)')
axes[0].set_title('Valor do Pedido por Review Score')
plt.sca(axes[0])
plt.xticks([1, 2, 3, 4, 5])

# Média de valor por review score
avg_value_by_score = df_orders.groupby('review_score')['payment_value'].mean().sort_index()
axes[1].bar(avg_value_by_score.index, avg_value_by_score.values, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Review Score')
axes[1].set_ylabel('Valor Médio do Pedido (R$)')
axes[1].set_title('Valor Médio por Review Score')
axes[1].set_xticks([1, 2, 3, 4, 5])

plt.tight_layout()
try:
    plt.savefig('../docs/images/eda_value_vs_nps.png', dpi=300, bbox_inches='tight')
    print("✓ Gráfico salvo: docs/images/eda_value_vs_nps.png")
except Exception as e:
    print(f"Erro ao salvar gráfico: {e}")
plt.show()

# Correlação
correlation = df_orders[['payment_value', 'review_score']].corr().iloc[0, 1]
print(f"\nCorrelação (Valor vs NPS): {correlation:.3f}")

### 5.2 Prazo de Entrega vs Review Score

In [ ]:
plt.close('all')
# Análise de atraso
df_orders['is_delayed'] = df_orders['delivery_delay_days'] > 0

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# NPS por status de atraso
nps_by_delay = df_orders.groupby('is_delayed')['review_score'].mean()
axes[0].bar(['No prazo', 'Atrasado'], nps_by_delay.values, edgecolor='black', alpha=0.7, color=['green', 'red'])
axes[0].set_ylabel('Review Score Médio')
axes[0].set_title('NPS Médio por Status de Entrega')
axes[0].axhline(df_orders['review_score'].mean(), color='blue', linestyle='--', label='Média Geral')
axes[0].legend()

# Distribuição de dias de entrega
axes[1].hist(df_orders['delivery_days'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Dias para Entrega')
axes[1].set_ylabel('Frequência')
axes[1].set_title('Distribuição do Tempo de Entrega')
axes[1].axvline(df_orders['delivery_days'].mean(), color='red', linestyle='--', label='Média')
axes[1].legend()

plt.tight_layout()
try:
    plt.savefig('../docs/images/eda_delivery_vs_nps.png', dpi=300, bbox_inches='tight')
    print("✓ Gráfico salvo: docs/images/eda_delivery_vs_nps.png")
except Exception as e:
    print(f"Erro ao salvar gráfico: {e}")
plt.show()

print(f"\nTempo médio de entrega: {df_orders['delivery_days'].mean():.1f} dias")
print(f"Taxa de atraso: {(df_orders['is_delayed'].sum() / len(df_orders)) * 100:.1f}%")
print(f"\nImpacto do atraso no NPS:")
print(f"  No prazo: {nps_by_delay[False]:.2f}")
print(f"  Atrasado: {nps_by_delay[True]:.2f}")
print(f"  Diferença: {(nps_by_delay[False] - nps_by_delay[True]):.2f} ({((nps_by_delay[False] - nps_by_delay[True]) / nps_by_delay[False] * 100):.1f}%)")

## 6. Análise Temporal

### 6.1 Evolução de Pedidos e Receita

In [ ]:
plt.close('all')
# Agrupar por mês
df_orders['year_month'] = df_orders['order_purchase_timestamp'].dt.to_period('M')
monthly_metrics = df_orders.groupby('year_month').agg({
    'order_id': 'count',
    'payment_value': ['sum', 'mean'],
    'review_score': 'mean'
}).reset_index()

monthly_metrics.columns = ['month', 'orders', 'revenue', 'aov', 'avg_nps']
monthly_metrics['month'] = monthly_metrics['month'].dt.to_timestamp()

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Pedidos por mês
axes[0, 0].plot(monthly_metrics['month'], monthly_metrics['orders'], marker='o', linewidth=2)
axes[0, 0].set_xlabel('Mês')
axes[0, 0].set_ylabel('Número de Pedidos')
axes[0, 0].set_title('Evolução de Pedidos')
axes[0, 0].grid(True, alpha=0.3)

# Receita por mês
axes[0, 1].plot(monthly_metrics['month'], monthly_metrics['revenue'], marker='o', linewidth=2, color='green')
axes[0, 1].set_xlabel('Mês')
axes[0, 1].set_ylabel('Receita (R$)')
axes[0, 1].set_title('Evolução de Receita')
axes[0, 1].grid(True, alpha=0.3)

# AOV por mês
axes[1, 0].plot(monthly_metrics['month'], monthly_metrics['aov'], marker='o', linewidth=2, color='orange')
axes[1, 0].set_xlabel('Mês')
axes[1, 0].set_ylabel('AOV (R$)')
axes[1, 0].set_title('Evolução do Ticket Médio (AOV)')
axes[1, 0].grid(True, alpha=0.3)

# NPS por mês
axes[1, 1].plot(monthly_metrics['month'], monthly_metrics['avg_nps'], marker='o', linewidth=2, color='red')
axes[1, 1].set_xlabel('Mês')
axes[1, 1].set_ylabel('NPS Médio')
axes[1, 1].set_title('Evolução do NPS')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(4.0, color='green', linestyle='--', alpha=0.5, label='Meta: 4.0')
axes[1, 1].legend()

plt.tight_layout()
try:
    plt.savefig('../docs/images/eda_temporal_evolution.png', dpi=300, bbox_inches='tight')
    print("Gráfico salvo: docs/images/eda_temporal_evolution.png")
except Exception as e:
    print(f"Erro ao salvar gráfico: {e}")
plt.show()

### 6.2 Sazonalidade - Dia da Semana e Hora

In [ ]:
plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Por dia da semana
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
orders_by_day = df_orders['order_day_of_week'].value_counts().reindex(day_order)
axes[0].bar(range(7), orders_by_day.values, edgecolor='black', alpha=0.7)
axes[0].set_xticks(range(7))
axes[0].set_xticklabels(['Seg', 'Ter', 'Qua', 'Qui', 'Sex', 'Sáb', 'Dom'])
axes[0].set_xlabel('Dia da Semana')
axes[0].set_ylabel('Número de Pedidos')
axes[0].set_title('Pedidos por Dia da Semana')

# Por hora do dia
orders_by_hour = df_orders['order_hour'].value_counts().sort_index()
axes[1].plot(orders_by_hour.index, orders_by_hour.values, marker='o', linewidth=2)
axes[1].set_xlabel('Hora do Dia')
axes[1].set_ylabel('Número de Pedidos')
axes[1].set_title('Pedidos por Hora do Dia')
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks(range(0, 24, 2))

plt.tight_layout()
try:
    plt.savefig('../docs/images/eda_seasonality.png', dpi=300, bbox_inches='tight')
    print("✓ Gráfico salvo: docs/images/eda_seasonality.png")
except Exception as e:
    print(f"Erro ao salvar gráfico: {e}")
plt.show()

print(f"\nPico de vendas:")
print(f"  Dia da semana: {orders_by_day.idxmax()}")
print(f"  Hora do dia: {orders_by_hour.idxmax()}h")

## 7. Análise Geográfica

### 7.1 Pedidos e Receita por Estado

In [ ]:
plt.close('all')
# Métricas por estado
state_metrics = df_orders.groupby('customer_state').agg({
    'order_id': 'count',
    'payment_value': ['sum', 'mean'],
    'review_score': 'mean',
    'customer_unique_id': 'nunique'
}).reset_index()

state_metrics.columns = ['state', 'orders', 'revenue', 'aov', 'avg_nps', 'customers']
state_metrics = state_metrics.sort_values('revenue', ascending=False)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Top 10 estados por receita
top_10_states = state_metrics.head(10)
axes[0, 0].barh(top_10_states['state'], top_10_states['revenue'], edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Receita (R$)')
axes[0, 0].set_ylabel('Estado')
axes[0, 0].set_title('Top 10 Estados por Receita')
axes[0, 0].invert_yaxis()

# Top 10 estados por pedidos
axes[0, 1].barh(top_10_states['state'], top_10_states['orders'], edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_xlabel('Número de Pedidos')
axes[0, 1].set_ylabel('Estado')
axes[0, 1].set_title('Top 10 Estados por Pedidos')
axes[0, 1].invert_yaxis()

# AOV por estado (top 10)
axes[1, 0].barh(top_10_states['state'], top_10_states['aov'], edgecolor='black', alpha=0.7, color='green')
axes[1, 0].set_xlabel('AOV (R$)')
axes[1, 0].set_ylabel('Estado')
axes[1, 0].set_title('Ticket Médio por Estado (Top 10)')
axes[1, 0].invert_yaxis()

# NPS por estado (top 10)
axes[1, 1].barh(top_10_states['state'], top_10_states['avg_nps'], edgecolor='black', alpha=0.7, color='red')
axes[1, 1].set_xlabel('NPS Médio')
axes[1, 1].set_ylabel('Estado')
axes[1, 1].set_title('Satisfação do Cliente por Estado (Top 10)')
axes[1, 1].axvline(4.0, color='green', linestyle='--', alpha=0.5, label='Meta: 4.0')
axes[1, 1].legend()
axes[1, 1].invert_yaxis()

plt.tight_layout()
try:
    plt.savefig('../docs/images/eda_geographic_analysis.png', dpi=300, bbox_inches='tight')
    print("✓ Gráfico salvo: docs/images/eda_geographic_analysis.png")
except Exception as e:
    print(f"Erro ao salvar gráfico: {e}")
plt.show()

print(f"\nTop 3 estados por receita:")
for i, row in state_metrics.head(3).iterrows():
    print(f"  {row['state']}: R$ {row['revenue']:,.2f} ({row['orders']} pedidos)")

print(f"\nMaior ticket médio: {state_metrics.loc[state_metrics['aov'].idxmax(), 'state']} (R$ {state_metrics['aov'].max():.2f})")
print(f"Melhor NPS: {state_metrics.loc[state_metrics['avg_nps'].idxmax(), 'state']} ({state_metrics['avg_nps'].max():.2f})")

### 7.2 Concentração Geográfica - Curva de Pareto

In [ ]:
plt.close('all')
# Curva de Pareto para concentração geográfica
state_metrics_pareto = state_metrics.sort_values('revenue', ascending=False).reset_index(drop=True)
state_metrics_pareto['cumulative_pct'] = (state_metrics_pareto['revenue'].cumsum() / state_metrics_pareto['revenue'].sum()) * 100

fig, ax1 = plt.subplots(figsize=(14, 8))

# Barras de receita
bars = ax1.bar(range(len(state_metrics_pareto)), state_metrics_pareto['revenue'], 
               color='skyblue', edgecolor='black', alpha=0.7, label='Receita por Estado')
ax1.set_xlabel('Estados (Ordenados por Receita)')
ax1.set_ylabel('Receita (R$)', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')

# Linha de Pareto
ax2 = ax1.twinx()
ax2.plot(range(len(state_metrics_pareto)), state_metrics_pareto['cumulative_pct'], 
         color='red', marker='o', linewidth=2, markersize=4, label='Curva de Pareto')
ax2.set_ylabel('Percentual Acumulado (%)', color='red')
ax2.tick_params(axis='y', labelcolor='red')
ax2.axhline(80, color='green', linestyle='--', alpha=0.7, label='80%')
ax2.axhline(90, color='orange', linestyle='--', alpha=0.7, label='90%')

# Configurações do gráfico
plt.title('Curva de Pareto - Concentração Geográfica da Receita')
plt.xticks(range(len(state_metrics_pareto)), state_metrics_pareto['state'], rotation=45)

# Adicionar anotações
top_80 = state_metrics_pareto[state_metrics_pareto['cumulative_pct'] <= 80]
top_90 = state_metrics_pareto[state_metrics_pareto['cumulative_pct'] <= 90]

ax2.annotate(f'80% da receita → {len(top_80)} estados', 
            xy=(len(top_80)-1, 80), xytext=(len(top_80)+2, 75),
            arrowprops=dict(arrowstyle='->', color='green'), color='green')

ax2.annotate(f'90% da receita → {len(top_90)} estados', 
            xy=(len(top_90)-1, 90), xytext=(len(top_90)+2, 85),
            arrowprops=dict(arrowstyle='->', color='orange'), color='orange')

fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
plt.tight_layout()

try:
    plt.savefig('../docs/images/eda_pareto_geographic.png', dpi=300, bbox_inches='tight')
    print("Gráfico salvo: docs/images/eda_pareto_geographic.png")
except Exception as e:
    print(f"Erro ao salvar gráfico: {e}")
plt.show()

print(f"\nCONCENTRAÇÃO GEOGRÁFICA:")
print(f"   • 80% da receita vem de {len(top_80)} estado(s): {', '.join(top_80['state'].tolist())}")
print(f"   • 90% da receita vem de {len(top_90)} estado(s)")
print(f"   • Concentração: {(len(top_80) / len(state_metrics) * 100):.1f}% dos estados geram 80% da receita")

## 8. Insights e Conclusões 

In [ ]:
# Recalcular todas as variáveis necessárias para garantir consistência
orders_by_day = df_orders['order_day_of_week'].value_counts()
orders_by_hour = df_orders['order_hour'].value_counts()
nps_by_delay = df_orders.groupby('is_delayed')['review_score'].mean()
correlation = df_orders[['payment_value', 'review_score']].corr().iloc[0, 1]

# Calcular métricas de concentração geográfica
state_metrics_pareto = state_metrics.sort_values('revenue', ascending=False).reset_index(drop=True)
state_metrics_pareto['cumulative_pct'] = (state_metrics_pareto['revenue'].cumsum() / state_metrics_pareto['revenue'].sum()) * 100
top_80_states = state_metrics_pareto[state_metrics_pareto['cumulative_pct'] <= 80]

print("\n" + "="*80)
print("PRINCIPAIS INSIGHTS E CONCLUSÕES")
print("="*80)

# Calcular métricas chave
total_revenue = df_orders['payment_value'].sum()
total_orders = len(df_orders)
avg_aov = df_orders['payment_value'].mean()
avg_nps = df_orders['review_score'].mean()
delay_rate = (df_orders['is_delayed'].sum() / len(df_orders)) * 100
positive_reviews = ((df_orders['review_score'] >= 4).sum() / len(df_orders)) * 100

print(f"\nMÉTRICAS CHAVE:")
print(f"   • Receita Total: R$ {total_revenue:,.2f}")
print(f"   • Pedidos Analisados: {total_orders:,}")
print(f"   • Ticket Médio (AOV): R$ {avg_aov:.2f}")
print(f"   • NPS Médio: {avg_nps:.2f}/5.0")
print(f"   • Taxa de Atraso: {delay_rate:.1f}%")
print(f"   • Avaliações Positivas (4-5): {positive_reviews:.1f}%")

print(f"\nINSIGHTS DE PERFORMANCE:")
print(f"   • Crescimento: Período de {(df_orders['order_purchase_timestamp'].max() - df_orders['order_purchase_timestamp'].min()).days} dias analisados")
print(f"   • Sazonalidade: Pico de vendas às {orders_by_hour.idxmax()}h, principalmente às {orders_by_day.idxmax()}s")
print(f"   • Eficiência: Tempo médio de entrega: {df_orders['delivery_days'].mean():.1f} dias")

print(f"\nCOMPORTAMENTO DO CLIENTE:")
print(f"   • Impacto de Atrasos: NPS cai {((nps_by_delay[False] - nps_by_delay[True]) / nps_by_delay[False] * 100):.1f}% quando há atraso")
print(f"   • Valor vs Satisfação: Correlação de {correlation:.3f} entre valor e NPS")
print(f"   • Engajamento: {df_orders['customer_unique_id'].nunique():,} clientes únicos")

print(f"\nANÁLISE GEOGRÁFICA:")
top_state = state_metrics.iloc[0]
print(f"   • Estado Líder: {top_state['state']} ({top_state['orders']} pedidos, R$ {top_state['revenue']:,.0f})")
print(f"   • Maior AOV: {state_metrics.loc[state_metrics['aov'].idxmax(), 'state']} (R$ {state_metrics['aov'].max():.2f})")
print(f"   • Melhor NPS: {state_metrics.loc[state_metrics['avg_nps'].idxmax(), 'state']} ({state_metrics['avg_nps'].max():.2f})")
print(f"   • Concentração: {len(top_80_states)} estado(s) geram 80% da receita")

print(f"\nRECOMENDAÇÕES ESTRATÉGICAS:")
print(f"   1. Otimizar logística para reduzir {delay_rate:.1f}% de atrasos")
print(f"   2. Focar em {len(top_80_states)} estado(s) que geram 80% da receita")
print(f"   3. Melhorar experiência em horários de pico ({orders_by_hour.idxmax()}h)")
print(f"   4. Desenvolver estratégias para aumentar ticket médio (atual: R$ {avg_aov:.2f})")
print(f"   5. Implementar programa de fidelidade baseado em NPS")

print("\n" + "="*80)
print("ANÁLISE EXPLORATÓRIA CONCLUÍDA")
print("="*80)